## World Development Indicators - Analytics

In [30]:
import pandas as pd
import numpy as np
import time

datos = pd.read_csv("suramerica.csv").drop('Unnamed: 0', 1)
paises = datos['CountryCode'].drop_duplicates().values.tolist()

preprocessing = 'imput'
search = 'original'
year_init = 2012
year_range = 5
look_back = 3
years = range(year_init-year_range+1,year_init+1)[::-1]

agricultura     = open("Indicadores/iagricultura.txt").read().split(',')
ambiente        = open("Indicadores/iambiente.txt").read().split(',')
ayuda           = open("Indicadores/iayuda.txt").read().split(',')
ciencia         = open("Indicadores/iciencia.txt").read().split(',')
clima           = open("Indicadores/iclima.txt").read().split(',')
comercio        = open("Indicadores/icomercio.txt").read().split(',')
deuda           = open("Indicadores/ideuda.txt").read().split(',')
economia        = open("Indicadores/ieconomia.txt").read().split(',')
educacion       = open("Indicadores/ieducacion.txt").read().split(',')
energia         = open("Indicadores/ienergia.txt").read().split(',')
finanzas        = open("Indicadores/ifinanzas.txt").read().split(',')
genero          = open("Indicadores/igenero.txt").read().split(',')
infraestructura = open("Indicadores/iinfraestructura.txt").read().split(',')
pobreza         = open("Indicadores/ipobreza.txt").read().split(',')
privado         = open("Indicadores/iprivado.txt").read().split(',')
publico         = open("Indicadores/ipublico.txt").read().split(',')
salud           = open("Indicadores/isalud.txt").read().split(',')
social          = open("Indicadores/isocial.txt").read().split(',')
trabajo         = open("Indicadores/itrabajo.txt").read().split(',')
urbano          = open("Indicadores/iurbano.txt").read().split(',')


conjunto_nombre = ['Agricultura','Ambiente','Ayuda','Ciencia','Clima','Comercio','Deuda','Economia','Educacion',
                   'Energia','Finanzas','Genero','Infraestructura','Pobreza','Privado','Publico','Salud','Social',
                   'Trabajo','Urbano']
conjunto = [agricultura,ambiente,ayuda,ciencia,clima,comercio,deuda,economia,educacion,energia,finanzas,genero,
            infraestructura,pobreza,privado,publico,salud,social,trabajo,urbano]
datos.head()

,CountryName,CountryCode,IndicatorName,IndicatorCode,Year,Value
0,Antigua and Barbuda,ATG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,1960,126.144000
1,Antigua and Barbuda,ATG,Age dependency ratio (% of working-age populat...,SP.POP.DPND,1960,88.237117
2,Antigua and Barbuda,ATG,"Age dependency ratio, old (% of working-age po...",SP.POP.DPND.OL,1960,7.779958
3,Antigua and Barbuda,ATG,"Age dependency ratio, young (% of working-age ...",SP.POP.DPND.YG,1960,80.457159
4,Antigua and Barbuda,ATG,"Birth rate, crude (per 1,000 people)",SP.DYN.CBRT.IN,1960,32.920000


In [31]:
def r2(y_true, y_predict):
    from sklearn.metrics import r2_score
    return r2_score(y_true, y_predict)

In [32]:
def tabla_base(indicadores):
    tab = pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False])
    return tab

In [33]:
def tabla_2_base(indicadores,look_back):
    temp_table = []
    for i in range(look_back):      
        temp_years = range(year_init-year_range-i+1,year_init-i+1)[::-1]
        temp_table.append(pd.DataFrame.pivot_table(datos, values='Value', index=['CountryCode', 'Year'], columns=['IndicatorCode']).loc[(paises,temp_years),indicadores].sortlevel(["CountryCode","Year"], ascending=[True,False]))
    return pd.DataFrame(np.column_stack(temp_table))

In [34]:
def estimator_Universal(estimador, X_train, X_test, y_train, y_test):
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.svm import SVR
    
    if(search=='original'):
        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor()
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1)
        if(estimador=='SVR'):
            estimator = SVR()
    else:
        best_params = SearchCV_Universal(estimador, search, X_train, y_train)       

        if(estimador=='DTR'):
            estimator = DecisionTreeRegressor().set_params(**best_params)
        if(estimador=='RFR'):
            estimator = RandomForestRegressor(n_jobs=-1).set_params(**best_params)
        if(estimador=='SVR'):
            estimator = SVR().set_params(**best_params)
        
    estimator.fit(X_train,y_train)
    y_predict = estimator.predict(X_test)
        
    return r2(y_test,y_predict)

In [35]:
def Preprocess(tab1,tab2,y_indicator):
    from sklearn.preprocessing import Imputer
    from sklearn.preprocessing import StandardScaler
    from sklearn.model_selection import train_test_split
       
    #Fusionamos la tabla_1 y el indicador y de la tabla_2
    tab_fusion = np.column_stack((np.array(tab1)[:,:],np.array(tab2)[:,y_indicator]))    
    
    #Eliminamos las columnas de NaN descartando Indicadores que no tienen regristros para ningún pais y año deseados
    df = pd.DataFrame(tab_fusion)
    df = df.dropna(how='all',axis=1)
    
    #Eliminamos las filas Si el valor a predecir es NaN
    df = df.dropna(subset=[df.iloc[:,-1].name])
    
    if(preprocessing=='zeros'):
        #Imputamos los NaN por Zero
        df = df.fillna(0)      
        
    if(preprocessing=='imput'):
        #Imputamos los NaN por la media de cada Indicador respectivamente      
        impute=Imputer(missing_values="NaN",strategy='mean',axis=0)
        impute.fit(df)
        df = pd.DataFrame(impute.transform(df))
        
    # Asignamos X e y, eliminando los indicadores que se correlacionen más (coeficiente > 0.7)
    df_ = df.corr()
    X = df.drop(df_[df_.iloc[:,-1] > 0.7].index, axis=1)
    y = df.iloc[:,-1]

    # Normalizamos los datos
    sc = StandardScaler()
    df_norm = sc.fit_transform(np.column_stack([X,y]))
    X = df_norm[:,:-1]
    y = df_norm[:,-1]
    
    # Separamos Train y Test respectivamente para X e y
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    return X_train, X_test, y_train, y_test

In [36]:
def iter_Splitter_Optimus(tab1,tab2): 
    R2_global = list()
    for i in range(0,np.shape(tab2.dropna(how='all',axis=1))[1]):

        X_train, X_test, y_train, y_test = Preprocess(tab1, tab2, i)

        result = estimator_Universal('DTR', X_train, X_test, y_train, y_test)

        if(result < 0.9): 
            temp = estimator_Universal('SVR', X_train, X_test, y_train, y_test)
            if(temp < 0.9): 
                temp2 = estimator_Universal('RFR', X_train, X_test, y_train, y_test)
                if (temp2 > temp): 
                    result = temp2
            if(temp > result): 
                result = temp

        R2_global.append(result)
    return R2_global

In [37]:
def SearchCV_Universal(estimador, search, X_train, y_train):
    from sklearn.model_selection import GridSearchCV
    from sklearn.model_selection import RandomizedSearchCV
    from sklearn.model_selection import ShuffleSplit
    from sklearn.tree import DecisionTreeRegressor
    from sklearn.ensemble import RandomForestRegressor
    from sklearn.svm import SVR
        
    if(estimador=='DTR'):
        estimator  = DecisionTreeRegressor()
        param_grid = {  'max_depth': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10], 
                        'splitter': ['best', 'random']
                     }
        
    if(estimador=='RFR'):
        estimator  = RandomForestRegressor()       
        param_grid = { 
                        "n_estimators"      : [10,20,30,40],
                        "max_features"      : ["auto", "sqrt", "log2"],
                        "min_samples_split" : [2,4,8],
                        "bootstrap": [True, False],
                     }
    if(estimador=='SVR'):
        estimator  = SVR()
        param_grid ={
                        'gamma'  : ['auto', 1e-3, 1e-4],
                        'C'      : [1, 10, 100, 1000],            
                    }

    if (search=='random'):
        grid = RandomizedSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
    if (search=='grid'):
        grid = GridSearchCV(estimator, param_grid, n_jobs=-1, cv=ShuffleSplit(test_size=0.2))
        
    grid.fit(X_train, y_train)
    
    return grid.best_params_

In [38]:
def filtro(indicadores_1,indicadores_2):
    if(indicadores_1!=indicadores_2):
        df = pd.DataFrame(indicadores_2)
        indicadores_2 = np.array(df.loc[~df.ix[:,0].isin(indicadores_1)]).flatten()
    return indicadores_2

In [39]:
def iterador_global(indicadores_1,indicadores_2,look_back): 
    
    tab1 = tabla_2_base(indicadores_1,look_back)
    tab2 = tabla_base(filtro(indicadores_1,indicadores_2))
    
    return iter_Splitter_Optimus(tab1,tab2)

In [40]:
def porcent_result_reg(df,indicadores_1,indicadores_2):
    df_temp = df
    x = 0.0
    for i in range(len(df_temp)):
        if(df_temp[i] > 0.9):
            x=x+1
    porcent = x/len(df)
    df = pd.DataFrame(df)
    df[df < 0] = 0.0
    result = np.array(df.values)   
    reg = len(filtro(indicadores_1,indicadores_2))
    return porcent,result,reg

In [41]:
def resultados(df,indicador_1,indicador_2,look_back):
    #Buscamos Resultados en base al conjunto de indicadores Base, Target y Look Back
    temp = df[(df['Base']==indicador_1)&(df['Target']==indicador_2)&(df['Look Back'] == look_back)]['Results'].values[0]
        
    # Códigos de los indicadores con registros   
    for i in range(len(conjunto_nombre)):
        
        if indicador_1 == conjunto_nombre[i]:
            indicador1 = conjunto[i]
        if indicador_2 == conjunto_nombre[i]:
            indicador2 = conjunto[i]

            df = pd.DataFrame(indicador2)
            # Filtro indicadores repetidos
            if(indicador1!=indicador2):
                indicador2 = np.array(df.loc[~df.ix[:,0].isin(indicador1)]).flatten()
            df_zeros = tabla_base(indicador2).dropna(how='all',axis=1)
                
    icodes = df_zeros.columns.values

    # Buscamos Nombres de los Códigos de los Indicadores
    nombres = pd.DataFrame(datos).loc[:,['IndicatorName','IndicatorCode']].set_index('IndicatorCode').loc[icodes].drop_duplicates()

    # Concatenamos Resultados y Nombres, Reindexando en base a los nombres
    df_results = pd.DataFrame(np.column_stack([nombres,temp]),columns=['Indicadores','Resultados'])
    
    return df_results

In [42]:
def buscar_mejores(df):
    return df[df.Resultados > 0.9].reset_index(drop=True)

In [43]:
def graficar(df,look_back):
    import seaborn as sns
    import matplotlib.pyplot as plt
    df_temp = df[df['Look Back'] == 1].drop(['Time','Reg','Look Back','Results'], 1)
    df_temp = pd.DataFrame(np.array(pd.DataFrame.pivot_table(df_temp, index=["Base"], columns=["Target"])), index=[conjunto_nombre], columns=[conjunto_nombre])
    ax = sns.heatmap(df_temp)
    plt.title('Look back = '+str(look_back), size=15)
    return plt.show()

## Iteramos el Primer Conjunto de Indicadores

In [44]:
start_time = time.time()
resultado_base = []

for i in range(len(conjunto)):
    for j in range(1,look_back+1):
        start_time2 = time.time()
        porcent , results, reg = porcent_result_reg(iterador_global(conjunto[0],conjunto[i],j),conjunto[0],conjunto[i])
        resultado_base.append([conjunto_nombre[0],conjunto_nombre[i],j,reg,porcent,"%s" % (time.time() - start_time2),results])
        pd.DataFrame(resultado_base, columns=["Base","Target","Look Back","Reg","%","Time","Results"]).to_csv('log_base.csv')

df = pd.DataFrame(resultado_base, columns=["Base","Target","Look Back","Reg","%","Time","Results"])
print("--- %s seconds ---" % (time.time() - start_time))
df

--- 1568.85250711 seconds ---


,Base,Target,Look Back,Reg,%,Time,Results
0,Agricultura,Agricultura,1,48,0.431818,11.6884729862,"[[0.687537742053], [0.900378443506], [0.614478..."
1,Agricultura,Agricultura,2,48,0.431818,13.2573390007,"[[0.174532673266], [0.660778255239], [0.391688..."
2,Agricultura,Agricultura,3,48,0.340909,15.4104478359,"[[0.291871660521], [0.58819008933], [0.6960061..."
3,Agricultura,Ambiente,1,96,0.333333,19.3097500801,"[[0.53920055157], [0.897619760524], [0.3933869..."
4,Agricultura,Ambiente,2,96,0.320000,20.1595678329,"[[0.901457094985], [0.927849746719], [0.770388..."
5,Agricultura,Ambiente,3,96,0.333333,21.9700679779,"[[0.635172585734], [0.489086899584], [0.734489..."
6,Agricultura,Ayuda,1,71,0.151515,19.9317851067,"[[1.0], [0.529069198976], [0.797003810511], [0..."
7,Agricultura,Ayuda,2,71,0.166667,21.595774889,"[[1.0], [0.866438751599], [0.183285082672], [0..."
8,Agricultura,Ayuda,3,71,0.075758,24.5776338577,"[[1.0], [0.816869758832], [0.440114563719], [0..."
9,Agricultura,Ciencia,1,13,0.307692,5.5919508934,"[[0.757632650556], [0.684131813953], [0.0], [0..."


In [45]:
pd.DataFrame.pivot_table(df, index=["Base","Target","Reg"], columns=["Look Back"])

%                    
Look Back                               1         2         3
Base        Target          Reg                              
Agricultura Agricultura     48   0.431818  0.431818  0.340909
            Ambiente        96   0.333333  0.320000  0.333333
            Ayuda           71   0.151515  0.166667  0.075758
            Ciencia         13   0.307692  0.461538  0.153846
            Clima           67   0.563636  0.472727  0.436364
            Comercio        142  0.262411  0.234043  0.234043
            Deuda           229  0.241071  0.254464  0.196429
            Economia        258  0.408000  0.308000  0.336000
            Educacion       168  0.318966  0.301724  0.215517
            Energia         51   0.480000  0.320000  0.340000
            Finanzas        67   0.233333  0.250000  0.250000
            Genero          175  0.280576  0.237410  0.215827
            Infraestructura 41   0.439024  0.292683  0.292683
            Pobreza         23   0.200000  0.200000  0.050000
            Privado         163  0.275168  0.221477  0.154362
            Publico         97   0.437500  0.387500  0.325000
            Salud           150  0.416000  0.360000  0.312000
            Social          35   0.457143  0.342857  0.285714
            Trabajo         130  0.204724  0.165354  0.118110
            Urbano          24   0.611111  0.500000  0.333333

In [25]:
pd.DataFrame.pivot_table(df, index=["Base","Target","Reg"], columns=["Look Back"])

%                    
Look Back                               1         2         3
Base        Target          Reg                              
Agricultura Agricultura     48   0.386364  0.363636  0.272727
            Ambiente        96   0.306667  0.306667  0.200000
            Ayuda           71   0.166667  0.136364  0.136364
            Ciencia         13   0.538462  0.230769  0.153846
            Clima           67   0.509091  0.472727  0.436364
            Comercio        142  0.269504  0.241135  0.283688
            Deuda           229  0.272321  0.209821  0.245536
            Economia        258  0.444000  0.316000  0.308000
            Educacion       168  0.293103  0.232759  0.206897
            Energia         51   0.380000  0.340000  0.320000
            Finanzas        67   0.316667  0.233333  0.200000
            Genero          175  0.330935  0.258993  0.223022
            Infraestructura 41   0.414634  0.341463  0.365854
            Pobreza         23   0.050000  0.250000  0.200000
            Privado         163  0.228188  0.214765  0.181208
            Publico         97   0.450000  0.300000  0.375000
            Salud           150  0.376000  0.368000  0.320000
            Social          35   0.285714  0.342857  0.171429
            Trabajo         130  0.244094  0.133858  0.118110
            Urbano          24   0.500000  0.444444  0.555556

In [29]:
buscar_mejores(resultados(df,'Agricultura','Clima',1))

,Indicadores,Resultados
0,CO2 emissions (kg per 2011 PPP $ of GDP),0.968052
1,CO2 emissions (kt),0.983193
2,CO2 emissions (metric tons per capita),0.956644
3,CO2 emissions from gaseous fuel consumption (%...,0.991519
4,CO2 emissions from liquid fuel consumption (% ...,0.935915
5,CO2 emissions from solid fuel consumption (kt),0.916753
6,CO2 intensity (kg per kg of oil equivalent ene...,0.918037
7,CPIA public sector management and institutions...,0.968009
8,"Community health workers (per 1,000 people)",1
9,Electric power consumption (kWh per capita),0.992047


## Iteramos todos los Conjuntos de Indicadores

In [14]:
start_time = time.time()
resultado_global = []

for i in range(len(conjunto)):
    for j in range(len(conjunto)):
        for k in range(1,look_back+1):
            start_time2 = time.time()
            porcent , results, reg = porcent_result_reg(iterador_global(conjunto[i],conjunto[j],k),conjunto[i],conjunto[j])
            resultado_global.append([conjunto_nombre[i],conjunto_nombre[j],k,reg,porcent,"%s" % (time.time() - start_time2),results])
            pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"]).to_csv('log_global.csv')


df_global = pd.DataFrame(resultado_global, columns=["Base","Target","Look Back","Reg","%","Time","Results"])
print("--- %s seconds ---" % (time.time() - start_time))
df_global

--- 4078.49419498 seconds ---


,Base,Target,Look Back,Reg,%,Time,Results
0,Agricultura,Agricultura,1,48,0.454545,13.310461998,"[[0.48548813546], [0.690857253739], [0.3913832..."
1,Agricultura,Agricultura,2,48,0.409091,14.1224291325,"[[0.434884720859], [0.844933742508], [0.617876..."
2,Agricultura,Agricultura,3,48,0.295455,15.2726609707,"[[0.568561350477], [0.802053364554], [0.0], [0..."
3,Agricultura,Economia,1,258,0.444000,50.8847777843,"[[0.924818541918], [0.752935843531], [0.0], [0..."
4,Agricultura,Economia,2,258,0.344000,61.1079230309,"[[0.914114571247], [0.991738050425], [0.163406..."
5,Agricultura,Economia,3,258,0.316000,65.6675961018,"[[0.607722447006], [0.727239521005], [0.0], [0..."
6,Agricultura,Educacion,1,168,0.275862,31.4311909676,"[[0.392046472394], [0.574037422681], [0.813921..."
7,Agricultura,Educacion,2,168,0.241379,32.9754679203,"[[0.126174439864], [0.555398802453], [0.257942..."
8,Agricultura,Educacion,3,168,0.215517,36.5862679482,"[[0.601778866496], [0.54871281022], [0.5529374..."
9,Agricultura,Salud,1,150,0.336000,31.2811381817,"[[0.0], [0.940136697129], [0.915370629453], [0..."


## Resultados

In [15]:
pd.DataFrame.pivot_table(df_global, index=["Base","Target","Reg"], columns=["Look Back"])

%                    
Look Back                           1         2         3
Base        Target      Reg                              
Agricultura Agricultura 48   0.454545  0.409091  0.295455
            Deuda       229  0.290179  0.200893  0.227679
            Economia    258  0.444000  0.344000  0.316000
            Educacion   168  0.275862  0.241379  0.215517
            Salud       150  0.336000  0.360000  0.240000
Deuda       Agricultura 48   0.204545  0.136364  0.068182
            Deuda       229  0.178571  0.191964  0.174107
            Economia    234  0.323144  0.310044  0.235808
            Educacion   168  0.129310  0.034483  0.043103
            Salud       152  0.110236  0.086614  0.110236
Economia    Agricultura 46   0.238095  0.214286  0.190476
            Deuda       203  0.194030  0.159204  0.203980
            Economia    260  0.464286  0.460317  0.365079
            Educacion   168  0.103448  0.051724  0.051724
            Salud       152  0.188976  0.188976  0.165354
Educacion   Agricultura 48   0.204545  0.068182  0.159091
            Deuda       229  0.183036  0.156250  0.151786
            Economia    260  0.253968  0.226190  0.234127
            Educacion   168  0.094828  0.120690  0.051724
            Salud       149  0.152000  0.128000  0.136000
Salud       Agricultura 46   0.452381  0.380952  0.261905
            Deuda       229  0.232143  0.183036  0.191964
            Economia    260  0.412698  0.345238  0.325397
            Educacion   165  0.289474  0.333333  0.412281
            Salud       152  0.448819  0.409449  0.456693

In [ ]:
graficar(df_global,1)